In [1]:
import pickle
import torch
import sys
import os

sys.path.append('../')

from generate_embeddings_connect4 import get_embeddings_qvalues, min_max_normalization
from probe import ProbeDataset, train_probe, test_probe
from GPT.dataset import EpisodeDataset
from GPT.model import Config

In [2]:
token_to_idx = {(i, j): i * 7 + j + 1 for i in range(6) for j in range(7)}
token_to_idx['<pad>'] = 0  # Padding token

vocab_size = 43
block_size = 42 # Honestly this could probably be whatever
embed_size = 512
num_heads = 8
num_layers = 8
dropout = 0.1

In [3]:
path = ''

In [4]:
with open(os.path.join(path, r'1\training_data\training_games_125000.pkl'), 'rb') as f:
    agent1 = pickle.load(f)
with open(os.path.join(path, r'2\training_data\training_games_125000.pkl'), 'rb') as f:
    agent2 = pickle.load(f)
with open(os.path.join(path, r'3\training_data\training_games_125000.pkl'), 'rb') as f:
    agent3 = pickle.load(f)
with open(os.path.join(path, r'4\training_data\training_games_125000.pkl'), 'rb') as f:
    agent4 = pickle.load(f)
with open(os.path.join(path, r'5\training_data\training_games_125000.pkl'), 'rb') as f:
    agent5 = pickle.load(f)
with open(os.path.join(path, r'6\training_data\training_games_125000.pkl'), 'rb') as f:
    agent6 = pickle.load(f)
with open(os.path.join(path, r'7\training_data\training_games_125000.pkl'), 'rb') as f:
    agent7 = pickle.load(f)
with open(os.path.join(path, r'8\training_data\training_games_125000.pkl'), 'rb') as f:
    agent8 = pickle.load(f)

In [5]:
with open(os.path.join(path, r'1\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent1 = pickle.load(f)
with open(os.path.join(path, r'2\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent2 = pickle.load(f)
with open(os.path.join(path, r'3\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent3 = pickle.load(f)
with open(os.path.join(path, r'4\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent4 = pickle.load(f)
with open(os.path.join(path, r'5\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent5 = pickle.load(f)
with open(os.path.join(path, r'6\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent6 = pickle.load(f)
with open(os.path.join(path, r'7\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent7 = pickle.load(f)
with open(os.path.join(path, r'8\training_data\q_vals_games_125000.pkl'), 'rb') as f:
    qagent8 = pickle.load(f)

In [6]:
train_ratio = 0.8
valid_ratio = 0.1

d1 = len(agent1)
d2 = len(agent2)
d3 = len(agent3)
d4 = len(agent4)
d5 = len(agent5)
d6 = len(agent6)
d7 = len(agent7)
d8 = len(agent8)

train1 = agent1[:int(train_ratio * d1)]
valid1 = agent1[int(train_ratio * d1):int((train_ratio + valid_ratio) * d1) ]
test1 = agent1[int((train_ratio + valid_ratio) * d1): ]

train2 = agent2[:int(train_ratio * d2)]
valid2 = agent2[int(train_ratio * d2):int((train_ratio + valid_ratio) * d2) ]
test2 = agent2[int((train_ratio + valid_ratio) * d2): ]

train3 = agent3[:int(train_ratio * d3)]
valid3 = agent3[int(train_ratio * d3):int((train_ratio + valid_ratio) * d3)]
test3 = agent3[int((train_ratio + valid_ratio) * d3):]

train4 = agent4[:int(train_ratio * d4)]
valid4 = agent4[int(train_ratio * d4):int((train_ratio + valid_ratio) * d4)]
test4 = agent4[int((train_ratio + valid_ratio) * d4):]

train5 = agent5[:int(train_ratio * d5)]
valid5 = agent5[int(train_ratio * d5):int((train_ratio + valid_ratio) * d5)]
test5 = agent5[int((train_ratio + valid_ratio) * d5):]

train6 = agent6[:int(train_ratio * d6)]
valid6 = agent6[int(train_ratio * d6):int((train_ratio + valid_ratio) * d6)]
test6 = agent6[int((train_ratio + valid_ratio) * d6):]

train7 = agent7[:int(train_ratio * d7)]
valid7 = agent7[int(train_ratio * d7):int((train_ratio + valid_ratio) * d7)]
test7 = agent7[int((train_ratio + valid_ratio) * d7):]

train8 = agent8[:int(train_ratio * d8)]
valid8 = agent8[int(train_ratio * d8):int((train_ratio + valid_ratio) * d8)]
test8 = agent8[int((train_ratio + valid_ratio) * d8):]

In [7]:
q1 = len(agent1)
q2 = len(agent2)
q3 = len(agent3)
q4 = len(agent4)
q5 = len(agent5)
q6 = len(agent6)
q7 = len(agent7)
q8 = len(agent8)

qtrain1 = agent1[:int(train_ratio * q1)]
qvalid1 = agent1[int(train_ratio * q1):int((train_ratio + valid_ratio) * q1)]
qtest1 = agent1[int((train_ratio + valid_ratio) * q1):]

qtrain2 = agent2[:int(train_ratio * q2)]
qvalid2 = agent2[int(train_ratio * q2):int((train_ratio + valid_ratio) * q2)]
qtest2 = agent2[int((train_ratio + valid_ratio) * q2):]

qtrain3 = agent3[:int(train_ratio * q3)]
qvalid3 = agent3[int(train_ratio * q3):int((train_ratio + valid_ratio) * q3)]
qtest3 = agent3[int((train_ratio + valid_ratio) * q3):]

qtrain4 = agent4[:int(train_ratio * q4)]
qvalid4 = agent4[int(train_ratio * q4):int((train_ratio + valid_ratio) * q4)]
qtest4 = agent4[int((train_ratio + valid_ratio) * q4):]

qtrain5 = agent5[:int(train_ratio * q5)]
qvalid5 = agent5[int(train_ratio * q5):int((train_ratio + valid_ratio) * q5)]
qtest5 = agent5[int((train_ratio + valid_ratio) * q5):]

qtrain6 = agent6[:int(train_ratio * q6)]
qvalid6 = agent6[int(train_ratio * q6):int((train_ratio + valid_ratio) * q6)]
qtest6 = agent6[int((train_ratio + valid_ratio) * q6):]

qtrain7 = agent7[:int(train_ratio * q7)]
qvalid7 = agent7[int(train_ratio * q7):int((train_ratio + valid_ratio) * q7)]
qtest7 = agent7[int((train_ratio + valid_ratio) * q7):]

qtrain8 = agent8[:int(train_ratio * q8)]
qvalid8 = agent8[int(train_ratio * q8):int((train_ratio + valid_ratio) * q8)]
qtest8 = agent8[int((train_ratio + valid_ratio) * q8):]


In [8]:
# Subsample Sizes
s = 100000
n = 12500

train = train1[:s] + train2[:s] + train3[:s] + train4[:s] + train5[:s] + train6[:s] + train7[:s] + train8[:s]
valid = valid1[:n] + valid2[:n] + valid3[:n] + valid4[:n] + valid5[:n] + valid6[:n] + valid7[:n] + valid8[:n] 
test = test1[:n] + test2[:n] + test3[:n] + test4[:n] + test5[:n] + test6[:n] + test7[:n] + test8[:n]


qtrain = qtrain1[:s] + qtrain2[:s] + qtrain3[:s] + qtrain4[:s] + qtrain5[:s] + qtrain6[:s] + qtrain7[:s] + qtrain8[:s]
qvalid = qvalid1[:n] + qvalid2[:n] + qvalid3[:n] + qvalid4[:n] + qvalid5[:n] + qvalid6[:n] + qvalid7[:n] + qvalid8[:n] 
qtest = qtest1[:n] + qtest2[:n] + qtest3[:n] + qtest4[:n] + qtest5[:n] + qtest6[:n] + qtest7[:n] + qtest8[:n]

In [9]:
train_dataset = EpisodeDataset(train, token_to_idx)
valid_dataset = EpisodeDataset(valid, token_to_idx)
test_dataset = EpisodeDataset(test, token_to_idx)

In [10]:
config = Config(vocab_size, block_size, n_layer=num_layers, n_head=num_layers, n_embd=embed_size)

In [11]:
def training_pipeline(folder_path: str, num_samples: int, layers: list, linear, model_load_path, train_random):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for i in layers:

        curr_path = os.path.join(folder_path, f"Layer_{i}")

        if not os.path.exists(curr_path):
            os.makedirs(curr_path)
        
        print(f"Layer {i}")
        
        # Retreive Embeddings and Normalize Q-Values

        embed_train, qval_train = get_embeddings_qvalues(num_samples, train, qtrain, i, config, token_to_idx, model_load_path = model_load_path, data_path = 'sampled_q_vals_train.pkl')
        embed_valid, qval_valid = get_embeddings_qvalues(num_samples, valid, qvalid, i, config, token_to_idx, model_load_path = model_load_path, data_path = 'sampled_q_vals_valid.pkl')
        embed_test, qval_test = get_embeddings_qvalues(num_samples, test, qtest, i, config, token_to_idx, model_load_path = model_load_path, data_path = 'sampled_q_vals_test.pkl')

        qval_train_norm, min, max = min_max_normalization(qval_train)
        qval_valid_norm = min_max_normalization(qval_valid, min, max)
        qval_test_norm = min_max_normalization(qval_test, min, max)

        d = len(qval_train_norm[0])
        n = embed_train[0].shape[0]

        # Non-Random

        probe_dataset_train = ProbeDataset(embed_train, qval_train_norm)
        probe_dataset_valid = ProbeDataset(embed_valid, qval_valid_norm)
        probe_dataset_test = ProbeDataset(embed_test, qval_test_norm)

        print("\nTraining Normal Probe\n")
        model_path, train_loss, valid_loss = train_probe(probe_dataset_train, probe_dataset_valid, device = device, epochs = 50, params = (d, n), model_dir = os.path.join(curr_path, f"Non_Random_Layer_{i}"), linear=linear)

        with open(os.path.join(curr_path, "normal_train_loss"), 'wb') as f:
            pickle.dump(train_loss, f)
        with open(os.path.join(curr_path, "normal_valid_loss"), 'wb') as f:
            pickle.dump(valid_loss, f)

        test_loss = test_probe(probe_dataset_test, model_path, (d, n), device, linear)
        print(f"MSE Loss: {test_loss:.4f}")

        if train_random:

            # Random
        
            random_embeddings_train = [torch.randn(512, 1) for _ in range(len(embed_train))]
            random_embeddings_valid = [torch.randn(512, 1) for _ in range(len(embed_valid))]
            random_embeddings_test = [torch.randn(512, 1) for _ in range(len(embed_test))]
            
            random_dataset_train = ProbeDataset(random_embeddings_train, qval_train_norm)
            random_dataset_valid = ProbeDataset(random_embeddings_valid, qval_valid_norm)
            random_dataset_test = ProbeDataset(random_embeddings_test, qval_test_norm)
    
            print("\nTraining Random Probe\n")
            random_path, random_train_loss, random_valid_loss = train_probe(random_dataset_train, random_dataset_valid, device = device, epochs = 50, params = (d, n), model_dir = os.path.join(curr_path, f"Random_Layer_{i}"), linear=linear)
    
            with open(os.path.join(curr_path, "random_train_loss"), 'wb') as f:
                pickle.dump(random_train_loss, f)
            with open(os.path.join(curr_path, "random_valid_loss"), 'wb') as f:
                pickle.dump(random_valid_loss, f)
    
            rand_loss = test_probe(random_dataset_test, random_path, (d, n), device, linear)
            print(f"Random MSE Loss: {rand_loss:.4f}\n")  

In [12]:
training_pipeline(folder_path = 'Nonlinear_Probe', num_samples = 8, layers = list(range(1, 9)), linear = False, model_load_path = 'Model_12.pth', train_random = True)

Layer 1

Training Normal Probe

Best Epoch: 5, Min MSE Loss: 0.11358589092565267
MSE Loss: 0.1162

Training Random Probe

Best Epoch: 3, Min MSE Loss: 0.14671152819063624
Random MSE Loss: 0.1487

Layer 2

Training Normal Probe

Best Epoch: 9, Min MSE Loss: 0.11317466846774966
MSE Loss: 0.1144

Training Random Probe

Best Epoch: 3, Min MSE Loss: 0.14671152819063624
Random MSE Loss: 0.1487

Layer 3

Training Normal Probe

Best Epoch: 9, Min MSE Loss: 0.11355891705450623
MSE Loss: 0.1147

Training Random Probe

Best Epoch: 3, Min MSE Loss: 0.14671152819063624
Random MSE Loss: 0.1487

Layer 4

Training Normal Probe

Best Epoch: 14, Min MSE Loss: 0.11391765904622314
MSE Loss: 0.1138

Training Random Probe

Best Epoch: 3, Min MSE Loss: 0.14671152819063624
Random MSE Loss: 0.1487

Layer 5

Training Normal Probe

Best Epoch: 14, Min MSE Loss: 0.11385590993179523
MSE Loss: 0.1136

Training Random Probe

Best Epoch: 3, Min MSE Loss: 0.14671152819063624
Random MSE Loss: 0.1487

Layer 6

Training 